In [ ]:

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
from streamlit_folium import folium_static
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Page configuration
st.set_page_config(
    page_title="Climate & Disaster Management System",
    page_icon="🌪️",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for better styling
st.markdown("""
<style>
    .main-header {
        font-size: 3rem;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 2rem;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.1);
    }
    .metric-card {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 1rem;
        border-radius: 10px;
        color: white;
        margin: 0.5rem 0;
    }
    .warning-box {
        background-color: #ff6b6b;
        padding: 1rem;
        border-radius: 10px;
        color: white;
        margin: 1rem 0;
    }
    .safe-box {
        background-color: #51cf66;
        padding: 1rem;
        border-radius: 10px;
        color: white;
        margin: 1rem 0;
    }
    .sidebar .sidebar-content {
        background: linear-gradient(135deg, #74b9ff 0%, #0984e3 100%);
    }
</style>
""", unsafe_allow_html=True)

# Load data function with caching
@st.cache_data
def load_disaster_data():
    """Load and preprocess disaster data"""
    try:
        # Load the CSV file
        df = pd.read_csv('top_100_disaster_data.csv')
        df['time'] = pd.to_datetime(df['time'])
        df['date'] = df['time'].dt.date
        df['hour'] = df['time'].dt.hour
        df['day_of_week'] = df['time'].dt.day_name()

        # Calculate movement metrics
        df_sorted = df.sort_values('time').reset_index(drop=True)
        df_sorted['lat_diff'] = df_sorted['latitude'].diff()
        df_sorted['lon_diff'] = df_sorted['longitude.anon'].diff()
        df_sorted['time_diff'] = df_sorted['time'].diff().dt.total_seconds() / 3600

        # Calculate approximate distance
        df_sorted['distance_km'] = np.sqrt(
            (df_sorted['lat_diff'] * 111) ** 2 + 
            (df_sorted['lon_diff'] * 85) ** 2
        )

        # Calculate speed
        df_sorted['speed_kmh'] = df_sorted['distance_km'] / df_sorted['time_diff']
        df_sorted['speed_kmh'] = df_sorted['speed_kmh'].replace([np.inf, -np.inf], np.nan)

        return df_sorted
    except Exception as e:
        st.error(f"Error loading data: {str(e)}")
        return None

def create_risk_assessment(df, current_time=None):
    """Generate risk assessment based on movement patterns"""
    if current_time is None:
        current_time = df['time'].max()

    # Recent activity analysis
    recent_data = df[df['time'] >= (current_time - timedelta(hours=6))]

    risk_factors = {
        'movement_speed': 0,
        'location_risk': 0,
        'frequency_risk': 0,
        'time_risk': 0
    }

    # Analyze movement speed
    avg_speed = recent_data['speed_kmh'].mean()
    if not pd.isna(avg_speed):
        if avg_speed > 50:  # High speed movement
            risk_factors['movement_speed'] = 0.8
        elif avg_speed > 20:
            risk_factors['movement_speed'] = 0.5
        else:
            risk_factors['movement_speed'] = 0.2

    # Location clustering risk
    lat_std = recent_data['latitude'].std()
    lon_std = recent_data['longitude.anon'].std()
    if lat_std < 0.01 and lon_std < 0.01:
        risk_factors['location_risk'] = 0.3  # Low movement, might be trapped
    else:
        risk_factors['location_risk'] = 0.1

    # Frequency risk
    data_points = len(recent_data)
    if data_points > 10:
        risk_factors['frequency_risk'] = 0.6  # High frequency might indicate distress
    elif data_points < 2:
        risk_factors['frequency_risk'] = 0.4  # Too few updates

    # Time-based risk (night hours are riskier)
    current_hour = current_time.hour
    if 0 <= current_hour <= 5 or 22 <= current_hour <= 23:
        risk_factors['time_risk'] = 0.6
    else:
        risk_factors['time_risk'] = 0.2

    overall_risk = np.mean(list(risk_factors.values()))

    return overall_risk, risk_factors

def generate_safety_recommendations(risk_level, risk_factors):
    """Generate safety recommendations based on risk assessment"""
    recommendations = []

    if risk_level > 0.7:
        recommendations.extend([
            "🚨 HIGH RISK DETECTED - Seek immediate shelter",
            "📱 Contact emergency services if in danger",
            "🏠 Move to the nearest evacuation center",
            "🔋 Ensure communication devices are charged"
        ])
    elif risk_level > 0.4:
        recommendations.extend([
            "⚠️ MODERATE RISK - Stay alert and prepared",
            "📦 Prepare emergency supplies",
            "📍 Identify nearest safe locations",
            "👥 Stay in contact with family/authorities"
        ])
    else:
        recommendations.extend([
            "✅ LOW RISK - Continue normal precautions",
            "📱 Keep monitoring weather updates",
            "🎒 Maintain emergency kit readiness",
            "🔄 Regular check-ins recommended"
        ])

    # Add specific recommendations based on risk factors
    if risk_factors['movement_speed'] > 0.5:
        recommendations.append("🚗 Reduce travel speed for safety")

    if risk_factors['location_risk'] > 0.4:
        recommendations.append("🚶 Consider changing location if possible")

    return recommendations

def create_movement_map(df):
    """Create an interactive map showing movement patterns"""
    center_lat = df['latitude'].mean()
    center_lon = df['longitude.anon'].mean()

    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    # Add movement path
    coordinates = [[row['latitude'], row['longitude.anon']] for _, row in df.iterrows()]

    folium.PolyLine(
        coordinates,
        weight=3,
        color='blue',
        opacity=0.8,
        popup='Movement Path'
    ).add_to(m)

    # Add markers for start and end points
    folium.Marker(
        [df.iloc[0]['latitude'], df.iloc[0]['longitude.anon']],
        popup='Start Point',
        icon=folium.Icon(color='green', icon='play')
    ).add_to(m)

    folium.Marker(
        [df.iloc[-1]['latitude'], df.iloc[-1]['longitude.anon']],
        popup='Current/Last Position',
        icon=folium.Icon(color='red', icon='stop')
    ).add_to(m)

    # Add high-risk areas (simulated based on data clustering)
    risk_areas = [
        {'lat': center_lat + 0.02, 'lon': center_lon + 0.02, 'risk': 'High'},
        {'lat': center_lat - 0.01, 'lon': center_lon - 0.01, 'risk': 'Medium'}
    ]

    for area in risk_areas:
        color = 'red' if area['risk'] == 'High' else 'orange'
        folium.CircleMarker(
            location=[area['lat'], area['lon']],
            radius=200,
            popup=f"{area['risk']} Risk Area",
            color=color,
            fill=True,
            fillOpacity=0.3
        ).add_to(m)

    return m

# Main Application
def main():
    st.markdown('<h1 class="main-header">🌪️ Climate & Disaster Management System</h1>', unsafe_allow_html=True)

    # Sidebar
    st.sidebar.markdown("## 📊 Dashboard Controls")

    # Load data
    with st.spinner("Loading disaster data..."):
        df = load_disaster_data()

    if df is None:
        st.error("Failed to load data. Please check the data file.")
        return

    # Sidebar options
    analysis_type = st.sidebar.selectbox(
        "Select Analysis Type",
        ["Overview Dashboard", "Movement Tracking", "Risk Assessment", "Temporal Analysis", "Emergency Response"]
    )

    time_filter = st.sidebar.selectbox(
        "Time Period",
        ["All Time", "Last 24 Hours", "Last 7 Days", "Custom Range"]
    )

    # Filter data based on time selection
    filtered_df = df.copy()
    if time_filter == "Last 24 Hours":
        cutoff_time = df['time'].max() - timedelta(days=1)
        filtered_df = df[df['time'] >= cutoff_time]
    elif time_filter == "Last 7 Days":
        cutoff_time = df['time'].max() - timedelta(days=7)
        filtered_df = df[df['time'] >= cutoff_time]
    elif time_filter == "Custom Range":
        start_date = st.sidebar.date_input("Start Date", df['time'].min().date())
        end_date = st.sidebar.date_input("End Date", df['time'].max().date())
        filtered_df = df[(df['time'].dt.date >= start_date) & (df['time'].dt.date <= end_date)]

    # Main content based on selection
    if analysis_type == "Overview Dashboard":
        show_overview_dashboard(filtered_df)
    elif analysis_type == "Movement Tracking":
        show_movement_tracking(filtered_df)
    elif analysis_type == "Risk Assessment":
        show_risk_assessment(filtered_df)
    elif analysis_type == "Temporal Analysis":
        show_temporal_analysis(filtered_df)
    elif analysis_type == "Emergency Response":
        show_emergency_response(filtered_df)

def show_overview_dashboard(df):
    """Display the main overview dashboard"""
    st.markdown("## 📊 Disaster Overview Dashboard")

    # Key metrics
    col1, col2, col3, col4 = st.columns(4)

    with col1:
        total_points = len(df)
        st.metric("Total Data Points", total_points, delta=None)

    with col2:
        duration = (df['time'].max() - df['time'].min()).days
        st.metric("Monitoring Duration", f"{duration} days", delta=None)

    with col3:
        avg_distance = df['distance_km'].mean()
        st.metric("Avg Movement Distance", f"{avg_distance:.1f} km", delta=None)

    with col4:
        risk_level, _ = create_risk_assessment(df)
        risk_color = "🔴" if risk_level > 0.7 else "🟡" if risk_level > 0.4 else "🟢"
        st.metric("Current Risk Level", f"{risk_color} {risk_level:.1%}", delta=None)

    # Charts
    col1, col2 = st.columns(2)

    with col1:
        # Daily activity chart
        daily_counts = df.groupby('date').size().reset_index(name='count')
        fig = px.line(daily_counts, x='date', y='count', title='Daily Activity Pattern',
                     labels={'date': 'Date', 'count': 'Number of Data Points'})
        fig.update_layout(height=400)
        st.plotly_chart(fig, use_container_width=True)

    with col2:
        # Hourly pattern
        hourly_counts = df.groupby('hour').size().reset_index(name='count')
        fig = px.bar(hourly_counts, x='hour', y='count', title='Hourly Activity Distribution',
                    labels={'hour': 'Hour of Day', 'count': 'Activity Count'})
        fig.update_layout(height=400)
        st.plotly_chart(fig, use_container_width=True)

def show_movement_tracking(df):
    """Display movement tracking analysis"""
    st.markdown("## 🗺️ Movement Tracking & Analysis")

    # Movement map
    st.subheader("Interactive Movement Map")
    movement_map = create_movement_map(df)
    folium_static(movement_map, width=1200, height=600)

    # Movement statistics
    col1, col2 = st.columns(2)

    with col1:
        st.subheader("Movement Statistics")
        total_distance = df['distance_km'].sum()
        max_distance = df['distance_km'].max()
        avg_speed = df['speed_kmh'].mean()

        st.write(f"**Total Distance Traveled:** {total_distance:.2f} km")
        st.write(f"**Maximum Single Movement:** {max_distance:.2f} km")
        st.write(f"**Average Speed:** {avg_speed:.2f} km/h")

        # Movement pattern analysis
        if df['distance_km'].std() > df['distance_km'].mean():
            st.warning("⚠️ Irregular movement patterns detected")
        else:
            st.success("✅ Normal movement patterns observed")

    with col2:
        # Speed over time chart
        speed_df = df.dropna(subset=['speed_kmh'])
        if not speed_df.empty:
            fig = px.scatter(speed_df, x='time', y='speed_kmh', 
                           title='Movement Speed Over Time',
                           labels={'time': 'Time', 'speed_kmh': 'Speed (km/h)'})
            fig.add_hline(y=speed_df['speed_kmh'].mean(), line_dash="dash", 
                         annotation_text="Average Speed")
            st.plotly_chart(fig, use_container_width=True)

def show_risk_assessment(df):
    """Display risk assessment dashboard"""
    st.markdown("## ⚠️ Risk Assessment Dashboard")

    # Calculate current risk
    risk_level, risk_factors = create_risk_assessment(df)
    recommendations = generate_safety_recommendations(risk_level, risk_factors)

    # Risk level display
    col1, col2, col3 = st.columns([2, 1, 2])

    with col1:
        st.subheader("Current Risk Assessment")
        if risk_level > 0.7:
            st.markdown(f'<div class="warning-box"><h3>🚨 HIGH RISK: {risk_level:.1%}</h3></div>', 
                       unsafe_allow_html=True)
        elif risk_level > 0.4:
            st.warning(f"⚠️ MODERATE RISK: {risk_level:.1%}")
        else:
            st.markdown(f'<div class="safe-box"><h3>✅ LOW RISK: {risk_level:.1%}</h3></div>', 
                       unsafe_allow_html=True)

    with col3:
        st.subheader("Risk Factors Breakdown")
        for factor, value in risk_factors.items():
            factor_name = factor.replace('_', ' ').title()
            st.progress(value, text=f"{factor_name}: {value:.1%}")

    # Recommendations
    st.subheader("Safety Recommendations")
    for rec in recommendations:
        st.write(f"• {rec}")

    # Risk trend analysis
    st.subheader("Risk Trend Analysis")

    # Calculate risk over time (simplified)
    risk_timeline = []
    for i in range(1, len(df)):
        subset = df.iloc[:i+1]
        risk, _ = create_risk_assessment(subset, subset['time'].iloc[-1])
        risk_timeline.append({'time': subset['time'].iloc[-1], 'risk': risk})

    if risk_timeline:
        risk_df = pd.DataFrame(risk_timeline)
        fig = px.line(risk_df, x='time', y='risk', title='Risk Level Over Time',
                     labels={'time': 'Time', 'risk': 'Risk Level'})
        fig.add_hline(y=0.7, line_dash="dash", line_color="red", 
                     annotation_text="High Risk Threshold")
        fig.add_hline(y=0.4, line_dash="dash", line_color="orange", 
                     annotation_text="Moderate Risk Threshold")
        st.plotly_chart(fig, use_container_width=True)

def show_temporal_analysis(df):
    """Display temporal analysis of the disaster data"""
    st.markdown("## 📈 Temporal Analysis")

    # Time series analysis
    col1, col2 = st.columns(2)

    with col1:
        # Activity by day of week
        dow_counts = df['day_of_week'].value_counts()
        fig = px.bar(x=dow_counts.index, y=dow_counts.values, 
                    title='Activity by Day of Week',
                    labels={'x': 'Day of Week', 'y': 'Activity Count'})
        st.plotly_chart(fig, use_container_width=True)

    with col2:
        # Movement distance over time
        distance_df = df[df['distance_km'] > 0]
        fig = px.scatter(distance_df, x='time', y='distance_km',
                        title='Movement Distance Over Time',
                        labels={'time': 'Time', 'distance_km': 'Distance (km)'})
        st.plotly_chart(fig, use_container_width=True)

    # Detailed timeline
    st.subheader("Detailed Activity Timeline")
    timeline_df = df[['time', 'latitude', 'longitude.anon', 'distance_km']].copy()
    timeline_df['distance_km'] = timeline_df['distance_km'].fillna(0)
    timeline_df = timeline_df.round(4)

    st.dataframe(timeline_df, use_container_width=True, height=400)

def show_emergency_response(df):
    """Display emergency response tools and information"""
    st.markdown("## 🚨 Emergency Response Center")

    # Emergency status
    risk_level, risk_factors = create_risk_assessment(df)

    col1, col2 = st.columns([2, 1])

    with col1:
        st.subheader("Emergency Action Plan")

        if risk_level > 0.7:
            st.error("🚨 EMERGENCY STATUS ACTIVE")
            emergency_actions = [
                "1. 📞 Call emergency services immediately (911/local emergency number)",
                "2. 🏃 Evacuate to nearest designated safe zone",
                "3. 📱 Send location to emergency contacts",
                "4. 🎒 Take emergency supplies if safely accessible",
                "5. 📻 Monitor emergency broadcasts for updates"
            ]
        elif risk_level > 0.4:
            st.warning("⚠️ ELEVATED ALERT STATUS")
            emergency_actions = [
                "1. 📦 Prepare emergency kit and supplies",
                "2. 📱 Charge all communication devices",
                "3. 🚗 Ensure vehicle is fueled and ready",
                "4. 👥 Establish contact with family/neighbors",
                "5. 📍 Identify evacuation routes and meeting points"
            ]
        else:
            st.success("✅ NORMAL STATUS - PREVENTIVE MEASURES")
            emergency_actions = [
                "1. 🎒 Maintain updated emergency kit",
                "2. 📱 Keep emergency contact list current",
                "3. 🏠 Review family emergency plan",
                "4. 📻 Stay informed about local weather/conditions",
                "5. 🔄 Regular system check and updates"
            ]

        for action in emergency_actions:
            st.write(action)

    with col2:
        st.subheader("Emergency Contacts")

        emergency_contacts = {
            "Emergency Services": "911",
            "Disaster Management": "1-800-DISASTER",
            "Red Cross": "1-800-RED-CROSS",
            "Local Emergency": "Local Number",
            "Family Contact": "Add Your Number"
        }

        for service, number in emergency_contacts.items():
            st.write(f"**{service}:** {number}")

        st.subheader("Quick Actions")

        if st.button("🚨 Report Emergency", type="primary"):
            st.success("Emergency reported! Location data sent to authorities.")

        if st.button("📍 Share Current Location"):
            last_location = df.iloc[-1]
            st.info(f"Location shared: {last_location['latitude']:.4f}, {last_location['longitude.anon']:.4f}")

        if st.button("📞 Auto-Call Emergency Services"):
            st.warning("This would initiate emergency call in a real implementation.")

    # Resource status
    st.subheader("Resource & Supply Status")

    col1, col2, col3, col4 = st.columns(4)

    resources = {
        "Water Supply": 0.8,
        "Food Supplies": 0.6,
        "Medical Kit": 0.9,
        "Communication": 0.7
    }

    for i, (resource, level) in enumerate(resources.items()):
        with [col1, col2, col3, col4][i]:
            st.metric(resource, f"{level:.0%}")
            if level < 0.3:
                st.error("Critical Low")
            elif level < 0.6:
                st.warning("Low")
            else:
                st.success("Good")

# Run the application
if __name__ == "__main__":
    main()
